In [8]:
def loadCoreArticle(string):    
    f = open('/home/lr/Downloads/conference_data/' + string + '.txt','r')
    sample = f.readlines()

    dict_finallis_keyword = eval(sample[0])
    f.close()


    return dict_finallis_keyword
"""
tochi_core = loadCoreArticle('tochi_core')
avi_core = loadCoreArticle('avi_core')
icmi_core = loadCoreArticle('icmi_core')
gi_core = loadCoreArticle('gi_core')
chi_core = loadCoreArticle('chi_core')
idc_core = loadCoreArticle('idc_core')
lak_review = loadCoreArticle('lak_review')
"""

cof = ['assets','avi','chi','cscw','dis','gi','group','icmi','idc','its','iui','lak','ls','mobilechi','tochi','ubicomp','uist']

In [9]:
import copy
title_list = []
reference_list = {}
for s in cof:
    name = s + "_core"
    
    data = loadCoreArticle(name)
    #print (s)
    #print (len(data))
    title_list.extend(list(data.keys()))
    for i in data:
        reference_list[i] = copy.deepcopy(data[i])

In [10]:
title_list[8] = ": designing a new class of computational toys"
title_list[16] = ": Haptic Feedback to Enhance Early Reading"
title_list[32] = ": The Effects of Curriculum-Aligned Making on Children's Self-Identity"

In [11]:
#获取数据
from elasticsearch import helpers
import elasticsearch


total_bibliometrics = {}
total_index_term = {}
total_reference = {}
total_author = {}
article_author = {}
total_article = {}
total_id = {}
total_conf = {}
total_year = {}
title_id = {}
total_group = {}

ES_SERVERS = [{
    # 'host': 'localhost',
    'host': '127.0.0.1',
    'port': 9200
}]

es_client = elasticsearch.Elasticsearch(
    hosts=ES_SERVERS
)

es_result = helpers.scan(
    client=es_client,
    query={
    "query": {
        "match_all": {}
    }
},
    scroll='5m',
    index='conference_total',
    #doc_type='conference_total',
    # timeout='1m'
)
# print(len(es_result))

#h_heads=[]
cont=0
for i in es_result:
    data_year = i['_source']
    h_head = i['_source']['h_head']
    id_num = i['_id']
    year = data_year['h_year']
    
    total_article[id_num] = {'title':[], 'abstract':[]}
    #if 'group' not in data_year.keys():
        #print (h_head)
           
    author = data_year['author']
    #reference = data_year['reference']
    index_term = data_year['index_term']
    bibliometrics = data_year['citition']
    title = data_year['title']
    abstract = data_year['abstract']
    reference = data_year['reference']
    group = data_year['group']
    #if reference == []:
        #print (id_num)
    
    cont += 1 
    #total_id.setdefault(id_num,[]).append(id_num)
    #total_bibliometrics.setdefault(id_num,[]).append(bibliometrics)
    total_reference[id_num] = reference
    #total_index_term.setdefault(id_num,[]).append(index_term)
    total_article[id_num]['title'] = title
    total_article[id_num]['abstract'] = abstract
    total_year[id_num] = year
    #article_author.setdefault(id_num,[]).append(author)
    total_conf[id_num] = h_head
    total_group[id_num] = group
    if title != []:
        title_id[title] = id_num
    
author_data = article_author
print (cont)

24648


In [260]:
print (total_article["914"])

{'title': 'Learning analytics beyond the LMS: the connected learning analytics toolkit', 'abstract': 'We present a Connected Learning Analytics (CLA) toolkit, which enables data to be extracted from social media and imported into a Learning Record Store (LRS), as defined by the new xAPI standard. A number of implementation issues are discussed, and a mapping that will enable the consistent storage and then analysis of xAPI verb/object/activity statements across different social media and online environments is introduced. A set of example learning activities are proposed, each facilitated by the Learning Analytics beyond the LMS that the toolkit enables.'}


# 引用网络

In [12]:
#一级引用网络
count_id = {}
total = 0
for s in total_reference:
    #print (total_reference[s])
    if total_reference[s] == []:
        total += 1
    for i in total_reference[s]:
        #print (i)
        if 'text' in i.keys():
            ref = i['text']
        elif 'title' in i.keys():
            ref = i['title']
            
        for m in title_list:
            if ref.find(m) >= 0:
                count = title_id[m]
                count_id.setdefault(s,[]).append(count)
                break

            
for s in count_id:
    count_id[s] = list(set(count_id[s]))

In [13]:
print (len(count_id))

1524


In [270]:
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, WordPunctTokenizer
from nltk.stem import WordNetLemmatizer
import numpy as np
import re
import math

#设置stopwords，缩略词拆解

stopwords = set (stopwords.words('english')+list(punctuation)+[').'])


#分段成句和分句成词
def splitSentence(paragraph):
    sent = sent_tokenize(paragraph)
    return sent


def wordtokenizer(sentence):
    word = []
    words = WordPunctTokenizer().tokenize(sentence)
    word = word + words
    return word

#文章预处理
def standardization(word_sent):
#转换为小写字母    
    text = []
    for s in word_sent:
        text.append(s.lower())
    
    return text

#词形还原
def lemmatizer(word):
    wnl = WordNetLemmatizer()
    text = []
#对单词类型标注
    text_tag = np.array(nltk.pos_tag(word))
    #print (text_tag)
    
    for s in text_tag:
        if s[1].startswith('N'):
            if s[1] == 'NNS' or s[1] == 'NN':
                text.append(wnl.lemmatize(s[0],'n'))
            else:
                text.append(s[0])
        elif s[1].startswith('V'):
            text.append(wnl.lemmatize(s[0], 'v'))
        elif s[1].startswith('J'):
            text.append(wnl.lemmatize(s[0], 'a'))
        elif s[1].startswith('R'):
            text.append(wnl.lemmatize(s[0], 'r'))
        else:
            text.append(s[0])
            
    #print (text)
    new_text = [word for word in text if word not in stopwords and 3<len(word)]
    
    return new_text

def filtrated(word):
    wnl = WordNetLemmatizer()
    text = []
#对单词类型标注
    text_tag = np.array(nltk.pos_tag(word))
    #print (text_tag)
    
    for s in text_tag:
        if s[1].startswith('N'):            
            text.append(s[0])
        
    
    return text

#统计词频
def wordFreq(word):
    word_Freq = {}
    for s in word:
        if not s in word_Freq:
            word_Freq[s] = 1
        else:
            word_Freq[s] += 1
    
    num = 0
    for key,value in word_Freq.items():
        num = num + value
    
    return word_Freq, num


In [262]:
#一级引用网络+教育筛选
count_id = {}
education_keywords = ['education','learn','teach','educate','e-learning','exam','student','curriculum','course','mooc']
total = 0
for s in total_reference:
    if total_article[s]['title'] == []:
        continue
    if total_article[s]['abstract'] != []:
        text = total_article[s]['title'] + " " + total_article[s]['abstract']
    else:
        text = total_article[s]['title']

    sent = splitSentence(text) 
    word_list = []
    for m in sent:
        word = wordtokenizer(m)                  
        word_list = word_list + word
    word_standard = standardization(word_list)
    word_ori = lemmatizer(word_standard)
    for j in word_ori:
        if j in education_keywords:       

            if total_reference[s] == []:
                total += 1
            for i in total_reference[s]:
                #print (i)
                if 'text' in i.keys():
                    ref = i['text']
                elif 'title' in i.keys():
                    ref = i['title']

                for m in title_list:
                    if ref.find(m) >= 0:
                        count = title_id[m]
                        count_id.setdefault(s,[]).append(count)
                        break

            break
            
for s in count_id:
    count_id[s] = list(set(count_id[s]))

In [268]:
article_list = list(count_id.keys())
#print (article_list)
for s in core_citing:
    if s not in article_list:
        article_list.append(s)
        
print (len(article_list))

891


In [49]:
title_1st = []
for s in count_id:
    #print (count_id[s])
    
    title = total_article[s]['title']
    
    title_1st.append(title)
            
print (len(title_1st))

1543


In [126]:
count_id = {}
total = 0
for s in total_reference:
    #print (total_reference[s])
    if total_reference[s] == []:
        total += 1
    for i in total_reference[s]:
        #print (i)
        if 'text' in i.keys():
            ref = i['text']
        elif 'title' in i.keys():
            ref = i['title']
            
        #count = 1
        for m in title_1st:
            if ref.find(m) >= 0:
                count = title_id[m]
                count_id.setdefault(s,[]).append(count)
                break
            #count += 1

In [127]:
print (len(count_id))

23747


# 建立引用网络

In [14]:
core_citing = {}
for s in count_id:
    #print (s)
    #print (count_id[s])
    for i in count_id[s]:
        core_citing.setdefault(i,[]).append(s)
        
print (len(core_citing))

160


In [23]:
print (core_citing.keys())

dict_keys(['67', '134', '289', '74', '361', '285', '352', '350', '298', '68', '360', '17966', '348', '212', '287', '283', '357', '351', '816', '7154', '9013', '5822', '612', '610', '18032', '7254', '17906', '11999', '3241', '2463', '291', '17844', '17750', '9157', '392', '661', '16980', '7252', '16979', '544', '17860', '999', '11696', '12383', '16891', '17968', '5413', '681', '644', '685', '934', '8290', '17907', '464', '6570', '439', '18954', '1796', '826', '2653', '5759', '920', '600', '856', '1012', '11110', '914', '1003', '603', '562', '3807', '819', '3731', '636', '445', '5744', '23470', '24154', '22393', '17153', '8925', '18011', '5821', '20741', '11208', '6338', '18977', '1843', '17579', '5917', '17747', '6682', '16876', '11155', '22403', '10035', '11640', '8958', '15208', '17885', '20971', '8651', '11071', '1732', '7014', '12687', '15031', '6293', '8249', '11083', '8752', '14781', '11170', '23147', '22752', '3366', '18295', '15590', '10606', '12702', '11228', '7190', '16992', '

In [15]:
for s in count_id:
    #print (len(count_id[s]))
    ids = list(set(count_id[s]))
    #print (len(ids))
    
    if len(count_id[s]) != len(ids):
        print (count_id[s])

In [16]:
for s in count_id:
    print (count_id[s])

['67']
['134', '289']
['74', '361', '285', '352', '67', '350', '289', '298', '68', '360', '17966', '348', '212', '287', '134', '283', '357', '351']
['816', '283']
['67']
['7154']
['9013']
['285']
['5822']
['350', '612', '610', '816', '18032', '283']
['298']
['7254', '17906']
['352', '11999', '3241']
['2463', '74']
['352']
['291', '17844']
['352', '350', '17750', '17966', '610', '212', '283']
['9157']
['392', '816', '283']
['289']
['289']
['67']
['661']
['350', '289', '360', '68', '18032', '134', '283']
['16980', '352', '610', '134', '7252']
['11999']
['283']
['298', '350']
['352']
['285', '16979', '283']
['285', '16979']
['74']
['610']
['285']
['610', '16979', '283']
['212']
['289']
['610', '360', '544']
['17860', '2463', '16980']
['11999']
['351']
['999']
['16979', '352', '350', '3241', '610', '351']
['610']
['16979', '352', '11696', '348', '287', '12383', '283']
['351']
['351']
['16891', '17968']
['610', '360']
['352']
['285']
['5413', '285', '16891', '17966']
['357']
['17966', '1796

In [40]:
import pandas as pd
import numpy as np

df = pd.DataFrame(columns = ['Source','Target','id','Type','weight'])
index = 0
Type = 'Undirected'
for s in count_id:
    for i in count_id[s]:
        temp = []
        if i in count_id.keys() and s in count_id[i]:
            print (s)
            print (i)
            continue
        
        #source = i
        #target = s
        
        temp.append(i)
        temp.append(s)
        temp.append(index)
        temp.append(Type)
        temp.append(1)
        
        df.loc[index] = temp
        
        index += 1
        

In [41]:
df.to_csv('edge.csv', sep=',', header=True, index=False)

In [19]:
num = 0
for s in df['Source']:
    #Target = df['Target']
    if s in df['Target']:
        
        num += 1
        
print (num)
        

0


In [44]:
onlineLearning = ['352', '600', '16891', '283', '392', '612', '287', '285', '610','16979', '16980', '18011', '17885', '17966', '350', '18977', '5917', '5744', '544']
tabletopInterface = ['8925', '12689', '3241', '11999', '15208', '10606', '11208', '22393', '11908', '12489', '23147', '22752', '10632', '15478', '18295', '11170', '14752']
STEMedu = ['11110', '14959', '18011', '74', '6570', '17747', '16394', '20230', '7252', '1732', '13840', '8181']
stuAnalysis = ['24154', '23470', '685', '661', '681', '999', '9013', '21556',  '1003', '16992', '17907']


In [43]:
print (total_article['610'])

{'title': 'Deconstructing disengagement: analyzing learner subpopulations in massive open online courses', 'abstract': 'As MOOCs grow in popularity, the relatively low completion rates of learners has been a central criticism. This focus on completion rates, however, reflects a monolithic view of disengagement that does not allow MOOC designers to target interventions or develop adaptive course features for particular subpopulations of learners. To address this, we present a simple, scalable, and informative classification method that identifies a small number of longitudinal engagement trajectories in MOOCs. Learners are classified based on their patterns of interaction with video lectures and assessments, the primary features of most MOOCs to date. In an analysis of three computer science MOOCs, the classifier consistently identifies four prototypical trajectories of engagement. The most notable of these is the learners who stay engaged through the course without taking assessments. 

In [24]:
for s in core_citing:
    #print (s)
    if s in onlineLearning:
        print (s)


285
352
350
17966
287
283
612
392
16980
16979
544
16891
600
5744
18011
18977
5917
17885


In [45]:
import math

df_node = pd.DataFrame(columns = ['id','Label','size','age'])
index = 0
for s in count_id:    
    temp = []
    #source = i
    #target = s
    
    weight = len(count_id[s])
    
    year = total_year[str(s)]
    if year == '2014/2015':
        year = 2014
    
    if total_group[s] == 'none':
        age = 30
    else:
        age = 2020 - int(year) + 1
    #print (age)

    temp.append(s)
    temp.append('')
    temp.append(weight)
    temp.append(age)
    
   

    df_node.loc[index] = temp

    index += 1

for s in core_citing:
    if s not in count_id.keys():
        temp = []
        weight = len(core_citing[s])
        year = total_year[str(s)]
        
        age = 2019 - int(year) + 1
        #print (age)
        
        
        temp.append(s)
        
        if str(s) in onlineLearning:
            temp.append("Online Learning" + s)
        elif str(s) in tabletopInterface:
            temp.append('Interface' + s)
        elif str(s) in STEMedu:
            temp.append('STEM' + s)
        elif str(s) in stuAnalysis:
            temp.append('analysis' + s)
        else:
            temp.append('')
        
        temp.append(weight)
        temp.append(age)
        #print (len(temp))
        
        
        
        df_node.loc[index] = temp
        index += 1
        


In [38]:
print (df_node.shape)

(1599, 4)


In [39]:
for s in df_node['Label']:
    print (s)

In [46]:
df_node.to_csv('node.csv', sep=',', header=True, index=False)

In [234]:
for s in df:
    print (s)

Source
Target
id
Type
weight


# 查找文章

In [172]:
def article(index):
    print (total_article[index]['title'])
    print (total_article[index]['abstract'])

In [244]:
dynamic_technology = ['11789','7014','1358','1714','3750','5845','5981','7907','8185','9617','9947','10160','10328','11535',
               '15184','17183','24153']

mental_health = ['24292','813','23608','10035','1351','1710','8233','8474','9358','9435','9615','9738','9950',
             '10482','18731','19973','23495','23504','23505','23536','23546','23547','23557','23572','23660',
             '23701','24004','24138','24164']

microtask = ['7494','6769','6812','8241','9211','9454','10286','10306','10366','18768']

stem11115 = ['11155','1443','4059','15153','6763','8648','9245','10494','10847','11464','13539','13679','14972',
             '15402','15563','15854','15883','15904','15943','17247','17492']

stem1732 = ['1732','1575','7973','8106','9704']

interface18433 = ['18433','2229','3088','5279','5306','5417','5923','10740','10926','11481','14656','14716','15809',
             '17220','17459','17515','20450','22590','23200','23440','24281']

interface11228 = ['11228','2250','3405','5355','12157','17053','17055','18492','18932','23892']

interface11170 = ['1796','331','11170','5515','5950','6728','8751','8866','9602','10498','10969','11707','11916',
             '15110','15884','16424','18393','18670','19295','20383','20472','20894','23211']

interface15346 = ['15346','8139','10410','10747','11592','13533','14918','14936','15172','15342','15396','15618',
             '15671','15708','15722','15758','15779','15785','15808','15842','15852','15927','16199','20460',
             '24219','24407','24631']

interface12689 = ['12689','3634','3889','4125','4290','4291','4652','4673','6284','8294','8749','8955','9135','9547',
             '10360','10816','12298','12576','13575','15438','15504','15898','17275','19645','21224','21389',
             '21768','21839','21962']
for s in stem11115:
    print (s)
    article(s)
    print ('\n')

11155
Storytelling alice motivates middle school girls to learn computer programming
We describe Storytelling Alice, a programming environment that introduces middle school girls to computer programming as a means to the end of creating 3D animated stories. Storytelling Alice supports story creation by providing 1) a set of high-level animations, that support the use of social characters who can interact with one another, 2) a collection of 3D characters and scenery designed to spark story ideas, and 3) a tutorial that introduces users to writing Alice programs using story-based examples. In a study comparing girls' experiences learning to program using Storytelling Alice and a version of Alice without storytelling support (Generic Alice), we found that users of Storytelling Alice and Generic Alice were equally successful at learning basic programming constructs. Participants found Storytelling Alice and Generic Alice equally easy to use and entertaining. Users of Storytelling Alice we

In [253]:
article("21694")
print ('\n')
article("18023")
print ('\n')
article("20368")
print ('\n')
#article("5821")

EmotiW 2018: Audio-Video, Student Engagement and Group-Level Affect Prediction
This paper details the sixth Emotion Recognition in the Wild (EmotiW) challenge. EmotiW 2018 is a grand challenge in the ACM International Conference on Multimodal Interaction 2018, Colarado, USA. The challenge aims at providing a common platform to researchers working in the affective computing community to benchmark their algorithms on 'in the wild' data. This year EmotiW contains three sub-challenges: a) Audio-video based emotion recognition; b) Student engagement prediction; and c) Group-level emotion recognition. The databases, protocols and baselines are discussed in detail.


MoodLight: Exploring Personal and Social Implications of Ambient Display of Biosensor Data
MoodLight is an interactive ambient lighting system that responds to biosensor input related to an individual's current level of arousal. Changes in levels of arousal correspond to fluctuations in the color of light provided by the system, 

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(columns = ['Source','Target','id','Type','weight'])
index = 0
Type = 'Undirected'
for s in count_id:
    for i in count_id[s]:
        temp = []
        if i in count_id.keys() and s in count_id[i]:
            print (s)
            print (i)
            continue
        
        #source = i
        #target = s
        
        temp.append(i)
        temp.append(s)
        temp.append(index)
        temp.append(Type)
        temp.append(1)
        
        df.loc[index] = temp
        
        index += 1
        

In [207]:
print(sorted(l))   

[14, 20, 32, 37, 41, 43, 43, 44, 46, 60, 66, 72, 85, 99, 100, 123, 143, 157, 162, 166, 182, 187, 195, 200, 215, 217, 219, 225, 226, 235, 236, 240, 242, 244, 247, 250, 255, 259, 260, 262, 263, 279, 281, 285, 293, 293, 295, 298, 303, 306, 319, 319, 331, 334, 349, 365, 373, 375, 400, 403, 406, 426, 430, 445, 459, 477, 490, 498, 515, 515, 521, 536, 545, 549, 565, 565, 569, 571, 586, 600, 605, 611, 622, 634, 647, 647, 647, 656, 657, 658, 671, 691, 696, 701, 708, 710, 711, 715, 718, 723]


In [207]:
print(sorted(l))   

[14, 20, 32, 37, 41, 43, 43, 44, 46, 60, 66, 72, 85, 99, 100, 123, 143, 157, 162, 166, 182, 187, 195, 200, 215, 217, 219, 225, 226, 235, 236, 240, 242, 244, 247, 250, 255, 259, 260, 262, 263, 279, 281, 285, 293, 293, 295, 298, 303, 306, 319, 319, 331, 334, 349, 365, 373, 375, 400, 403, 406, 426, 430, 445, 459, 477, 490, 498, 515, 515, 521, 536, 545, 549, 565, 565, 569, 571, 586, 600, 605, 611, 622, 634, 647, 647, 647, 656, 657, 658, 671, 691, 696, 701, 708, 710, 711, 715, 718, 723]


In [207]:
print(sorted(l))   

[14, 20, 32, 37, 41, 43, 43, 44, 46, 60, 66, 72, 85, 99, 100, 123, 143, 157, 162, 166, 182, 187, 195, 200, 215, 217, 219, 225, 226, 235, 236, 240, 242, 244, 247, 250, 255, 259, 260, 262, 263, 279, 281, 285, 293, 293, 295, 298, 303, 306, 319, 319, 331, 334, 349, 365, 373, 375, 400, 403, 406, 426, 430, 445, 459, 477, 490, 498, 515, 515, 521, 536, 545, 549, 565, 565, 569, 571, 586, 600, 605, 611, 622, 634, 647, 647, 647, 656, 657, 658, 671, 691, 696, 701, 708, 710, 711, 715, 718, 723]


In [207]:
print(sorted(l))   

[14, 20, 32, 37, 41, 43, 43, 44, 46, 60, 66, 72, 85, 99, 100, 123, 143, 157, 162, 166, 182, 187, 195, 200, 215, 217, 219, 225, 226, 235, 236, 240, 242, 244, 247, 250, 255, 259, 260, 262, 263, 279, 281, 285, 293, 293, 295, 298, 303, 306, 319, 319, 331, 334, 349, 365, 373, 375, 400, 403, 406, 426, 430, 445, 459, 477, 490, 498, 515, 515, 521, 536, 545, 549, 565, 565, 569, 571, 586, 600, 605, 611, 622, 634, 647, 647, 647, 656, 657, 658, 671, 691, 696, 701, 708, 710, 711, 715, 718, 723]


In [194]:
#print (total_article['15438'])
#print (total_article['13575'])
print (total_conf['15438'])
print (total_conf['13575'])
print (total_year['13575'])

IDC
ASSETS
2019
2019


# 生成词云

In [248]:
def generateWC(group, name):
    word_sum = []
    for s in group:
        
        title = total_article[s]['title']
        abstract = total_article[s]['abstract']
        text = title + ' ' + title + ' ' + abstract

        sent = splitSentence(title) 
        word_list = []
        for m in sent:
            word = wordtokenizer(m)                  
            word_list = word_list + word

        word_standard = standardization(word_list)
        word_ori = lemmatizer(word_standard)

        word_sum.extend(word_ori)

    #print (word_sum)
    prep_list = '/'.join(word_sum)
    #print (prep_list)
    
    from wordcloud import WordCloud
    wc = WordCloud(width = 800, height = 600).generate(prep_list)
    image = wc.to_image()
    image.show()
    wc.to_file(name + ".png")

In [250]:
#generateWC(dynamic_technology,"dynamic_technology")
#generateWC(mental_health,"mental_health")
#generateWC(microtask,"microtask")
#generateWC(stem1732,"special_edu")
generateWC(interface11170,"poor_edu")
generateWC(interface11228,'ubiquitous')
generateWC(interface12689,'agent')
generateWC(interface15346,'situation')

# 相似文章

In [254]:
title_list[8] = ": designing a new class of computational toys"
title_list[16] = ": Haptic Feedback to Enhance Early Reading"
title_list[32] = ": The Effects of Curriculum-Aligned Making on Children's Self-Identity"

In [255]:
#获取数据
from elasticsearch import helpers
import elasticsearch

similar_record = {}
ES_SERVERS = [{
    # 'host': 'localhost',
    'host': '39.97.190.184',
    'port': 9200
}]

es_client = elasticsearch.Elasticsearch(
    hosts=ES_SERVERS
)

es_result = helpers.scan(
    client=es_client,
    query={
    "query": {
        "match_all": {}
    }
},
    scroll='5m',
    index='conference_rec_total',
    #doc_type='conference_total',
    # timeout='1m'
)
# print(len(es_result))

#h_heads=[]
cont=0
for i in es_result:
    data_year = i['_source']
    #h_head = i['_source']['h_head']
    #id_num = i['_id']
    title = data_year['title']
    #print (title)
    if title in title_list:
        similar_record[title] = data_year['score_values']
    

In [32]:
print (len(similar_record))

164


In [37]:
sim_id = {}
count = 0
for s in similar_record:
    #print (similar_record[s])
    #print (len(similar_record[s]))
    for i in range(1,len(similar_record[s])):
        num = similar_record[s][i]['title_id']
        sim_id.setdefault(num,[]).append(count)
        
    count += 1
print (len(sim_id))

7477
